## PyBaMM Model Implementation

In [111]:
import pybamm
import numpy as np
from scipy import constants
import matplotlib.pyplot as plt
import os
# os.chdir(pybamm.__path__[0]+'/..')
# os.chdir('/Users/andrew/Dropbox/Research/DPhil/9 Pybamm_Modelling/PyBaMM')
# os.getcwd()

#### 1. Initialise model

In [112]:
model = pybamm.BaseModel()

#### 2. Define parameters and variables


In [113]:
# Defining Parameters

# PARAMETERS
#--------------------------------------------

# Galvanostatic Polarization (1. Pulse, 2. Relax)
# t_pulse = pybamm.Parameter("Current pulse time [s]")
# t_relax = pybamm.Parameter("Current off relaxation time [s]")
# def I_applied(t_pulse,t_relax):
#     "Current profile"
#     inputs =  {"Current pulse time [s]": t_pulse,
#                "Current off relaxation time [s]": t_relax,
#               }
#     return pybamm.FunctionParameter("Current profile [A]", inputs)

I_applied = pybamm.Scalar(0.002) #("Applied current [A]")

                                    
# Geometery
L = pybamm.Scalar(0.002) #("Electrolyte length [m]")
A = pybamm.Scalar(0.00005) #("Electrode area [m2]")

# Physical constants
R = pybamm.Scalar(constants.R) #("Gas constant [J.mol-1.K-1]")
F = pybamm.Scalar(constants.physical_constants["Faraday constant"][0]) #("Faradays constant [C.mol-1]")
T = pybamm.Scalar(298.15) #("Temperature [K]")

# Stoich and charge constants
nu_p = pybamm.Scalar(1) #("Cation formula unit")
nu_m = pybamm.Scalar(1) #("Anion formula unit")
z_p = pybamm.Scalar(1) #("Cation equivalent charge")
z_m = pybamm.Scalar(-1) #("Anion equivalent charge")
s_p = pybamm.Scalar(-1) #("Cation stoichiometric coefficient")
s_n = pybamm.Scalar(0) #("Anion stoichiometric coefficient")
s_0 = pybamm.Scalar(0) #("Solvent stoichiometric coefficient")

# Electrolyte bulk property
c_init = pybamm.Scalar(1000) #("Bulk electrolyte concentration [mol.m-3]")
M_e = pybamm.Scalar(0.151905) #("Salt molar mass [kg.mol-1]")
M_0 = pybamm.Scalar(0.1041) #("Solvent molar mass [kg.mol-1]")

# Separator property
Br = pybamm.Scalar(1.5) #("Bruggeman coefficient")
eta  = pybamm.Scalar(0.85) #("Porosity")


# CONSTANT FUNCTION PARAMETERS (for now)
tp0 = pybamm.Scalar(0.4) 
V_0 = pybamm.Scalar(0.0001) #m3/mol
V_e = pybamm.Scalar(0.00005) #m3/mol
Chi = pybamm.Scalar(1) 
K_eff = pybamm.Scalar(0.5) #S/m
D_eff = pybamm.Scalar(3E-10) #m2/s

In [114]:
# Defining variables
x = pybamm.SpatialVariable("x", domain="Electrolyte", coord_sys="cartesian")
c = pybamm.Variable("Salt concentration", domain="Electrolyte")
phi = pybamm.Variable("Potential", domain="Electrolyte")
p = pybamm.Variable("Dummy pressure", domain="Electrolyte")

#### 3. State governing equations

In [118]:
#Evaluate composition dependent parameters
tp0_eval = tp0
V_0_eval = V_0
V_e_eval = V_e
Chi_eval = Chi
K_eff_eval = K_eff
D_eff_eval = D_eff

nu = nu_p + nu_m
vbox = -pybamm.grad(p) #pybamm doesnt define vector variables, write in terms of scalar p

# MacInnes Equation (Hou 26) rewritten in terms of i_curr
i_curr =  (K_eff_eval* 
           (-pybamm.grad(phi) + 
            ((nu*R*T*Chi_eval*(1-tp0_eval))/(c*F*z_p*nu_p*(1+(nu*V_0_eval-V_e_eval)*c)))*pybamm.grad(c)))

# Define flux in terms of c (Hou 27) 
N_p = (-nu_p*D_eff_eval*pybamm.grad(c) 
               + (tp0_eval/(F*z_p))*i_curr 
                            + nu_p*c*vbox)

# Material balance (Hou 31) rearranged with time derivative on LHS
dcdt = (-1/eta)*pybamm.div(N_p/nu_p)

# Volume continuity (Hou 30) rearranged to = 0
vol_cont = (pybamm.div(vbox) 
            + pybamm.inner((V_e_eval/(F*z_p*nu_p))*i_curr, pybamm.grad(tp0_eval)) 
                + pybamm.inner((D_eff_eval/(1-(V_e_eval*c)))*pybamm.grad(c), pybamm.grad(V_e_eval)))



model.algebraic = {phi: pybamm.div(i_curr), p: vol_cont}
model.rhs = {c: dcdt}



model.initial_conditions = {
    c: c_init,
    phi: pybamm.Scalar(0), #arbitrary
    p: pybamm.Scalar(0), #arbitrary
}


# # Boundary conditions 

# Plug (Hou 33) into (Hou 27) and rearrange for dcdx term, terms collected here:
dcdx_left = pybamm.boundary_value((-1/(nu_p*D_eff_eval))*((I_applied/(F*z_p*A))*(1-tp0_eval)*(1-c*V_e_eval)),"left")
dcdx_right = pybamm.boundary_value((-1/(nu_p*D_eff_eval))*((I_applied/(F*z_p*A))*(1-tp0_eval)*(1-c*V_e_eval)),"right")

# Equation (Hou 26) evaluated at x=0
dphidx_left = pybamm.boundary_value((I_applied/A)/K_eff_eval
                -((nu*R*T*Chi_eval*(1-tp0_eval))/(c*F*z_p*nu_p*(1+(nu*V_0_eval-V_e_eval)*c)))*pybamm.grad(c),"left")

# Equation (Hou 33)
dpdx_left = pybamm.boundary_value(V_e_eval*(1-tp0_eval)*I_applied/(F*z_p*nu_p*A),"left") #dpdx is vbox at x=0

# dcdx_left = 1# c*V_e_eval
# dcdx_right = 1# c*V_e_eval
# dphidx_left = 1# (I_applied/A)/K_eff_eval
# dpdx_left = 1# V_e_eval*(1-tp0_eval)*I_applied/(F*z_p*nu_p*A)

#p and phi are algebraic variables so need dirichlet bcs either right or left, otherwise undefined up to a constant
model.boundary_conditions = {
    c: {"left": (dcdx_left, "Neumann"), "right": (dcdx_right, "Neumann")},
    phi: {"left": (dphidx_left, "Neumann"), "right": (0, "Dirichlet")},
    p: {"left": (dpdx_left, "Neumann"), "right": (0, "Dirichlet")},
    }

In [119]:
%%time

# Geometry
geometry = {"Electrolyte": {"primary": {x: {"min": pybamm.Scalar(0), "max": L}}}}
    
# Mesh and discretise
submesh_types = {"Electrolyte": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh)}
# submesh_types = {"Electrolyte": pybamm.MeshGenerator(pybamm.Exponential1DSubMesh)}
var_pts = {x: 1000}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

spatial_methods = {"Electrolyte": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)
disc.process_model(model);

ShapeError: Cannot find shape (original error: operands could not be broadcast together with shapes (1001,1) (999,1) )